<a href="https://colab.research.google.com/github/sadeelmu/deeplearning/blob/main/Rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 5a - RNN

This lab session will introduce you to RNNs with a toy problem. We have simulated 3 types of insect motions in 2D: *ant* (steady), *slug* (fluctuating) and *grasshopper* (explosive). 2D positions over time are given as (num_times, 2) arrays; the first row contains the x coordinates, the second row the y coordinates. These will be fed to an RNN, which you will train to recognize the three motion types.

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.animation as amt
from matplotlib import rc
import numpy as np
from timeit import default_timer as timer
import pickle
import random

rc('animation', html='jshtml')
random.seed(2111994)

## Download

In [ ]:
!wget --content-disposition https://seafile.unistra.fr/f/2fadadbc51b54de09c82/?dl=1

--2024-11-29 14:16:17--  https://seafile.unistra.fr/f/2fadadbc51b54de09c82/?dl=1
Resolving seafile.unistra.fr (seafile.unistra.fr)... 77.72.44.41
Connecting to seafile.unistra.fr (seafile.unistra.fr)|77.72.44.41|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://seafile.unistra.fr/seafhttp/files/bc7e16e4-55a8-46ea-a190-b698d8d776db/bugs.pickle [following]
--2024-11-29 14:16:18--  https://seafile.unistra.fr/seafhttp/files/bc7e16e4-55a8-46ea-a190-b698d8d776db/bugs.pickle
Reusing existing connection to seafile.unistra.fr:443.
HTTP request sent, awaiting response... 200 OK
Length: 21101550 (20M) [application/octet-stream]
Saving to: ‘bugs.pickle’

bugs.pickle         100%[===================>]  20.12M  52.6MB/s    in 0.4s    

2024-11-29 14:16:18 (52.6 MB/s) - ‘bugs.pickle’ saved [21101550/21101550]



## Dataset

Load it using the following cell:

In [ ]:
DATA_PATH = "bugs.pickle"

with open(DATA_PATH, "rb") as f:
    data = pickle.load(f)

x_train = data["x_train"]
y_train = data["y_train"]
x_val   =  data["x_val"]
y_val   =  data["y_val"]

x_train = [np.transpose(v) for v in x_train]
x_val = [np.transpose(v) for v in x_val]

This cell will load a few helpful functions for previewing.

In [ ]:
BBOX_MARGIN = 1.2
ANIM_REFRESH_TIME = 1000 / 60

def trajectory_bbox(positions):
  x_min = np.min(positions[0, :])
  x_max = np.max(positions[0, :])
  y_min = np.min(positions[1, :])
  y_max = np.max(positions[1, :])
  delta_x = x_max - x_min
  delta_y = y_max - y_min
  delta = BBOX_MARGIN * max(delta_x, delta_y) / 2
  cx = (x_min + x_max) / 2
  cy = (y_min + y_max) / 2
  bbox_x1 = cx - delta
  bbox_x2 = cx + delta
  bbox_y1 = cy - delta
  bbox_y2 = cy + delta
  return bbox_x1, bbox_x2, bbox_y1, bbox_y2


def plot_trajectory(positions, save_png=None):
  bbox = trajectory_bbox(positions)
  fig = plt.figure()
  ax = fig.add_subplot(111)
  ax.plot(positions[0, :], positions[1, :])
  ax.set_xlim(bbox[0], bbox[1])
  ax.set_ylim(bbox[2], bbox[3])
  ax.set_aspect(1.0)
  plt.show()


def animate_motion(positions, n_frames, save_gif=None):
  bbox = trajectory_bbox(positions)
  fig = plt.figure()
  ax = fig.add_subplot(111)
  handle = ax.scatter([], [], marker=".", color="r", alpha=1)

  def init():
    ax.set_xlim(bbox[0], bbox[1])
    ax.set_ylim(bbox[2], bbox[3])
    ax.set_aspect(1.0)
    return handle,

  def update(id_t):
    handle.set_offsets(positions[:, id_t])
    return handle,

  anim = amt.FuncAnimation(
    fig,
    update,
    frames=np.arange(n_frames),
    interval=ANIM_REFRESH_TIME,
    init_func=init,
    blit=True,
    repeat=False
  )
  return anim

#### QUESTION 1

Preview a sample here; change the value of ```ID_SAMPLE``` a few times to get familiar with the different motion types.

In [ ]:
ID_SAMPLE = 0
a = animate_motion(x_train[ID_SAMPLE], 200)
plt.close()

Animate:

In [ ]:
a

We will package all of this into tensorflow datasets. **Batching makes the input a (48, num_times, 2) tensor**.

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
train_dataset = train_dataset.shuffle(10000).batch(48)
val_dataset = val_dataset.batch(48)

## Model


We will use an RNN with an LSTM cell. Here is the definition; pay attention to the syntax: first we create the *cell*, then ask Tensorflow for a function that will call it over the elements of a sequence.

In [ ]:
class RNN_model(tf.keras.Model):
  def __init__(self, memory_size):
    super().__init__()
    self._cell = tf.keras.layers.LSTMCell(memory_size)
    self._rnn = tf.keras.layers.RNN(self._cell)
    self._dense = tf.keras.layers.Dense(
      units=N_CLASSES,
      activation="softmax"
    )

  def call(self, x):
    res = x
    res = self._rnn(x)
    res = self._dense(res)
    return res

## Training

Here is an optimizer for performing gradient descent, as well as the functions for one iteration of training and one iteration of evaluation.

*You have seen them before; those are very generic. Being comfortable with them is highly valuable for your general understanding of deep learning*

In [ ]:
optimizer = tf.keras.optimizers.RMSprop(0.005)

@tf.function
def train_step(inp, labels, model):
  with tf.GradientTape() as tape:
    outp = model(inp, training=True)
    loss = tf.reduce_mean(tf.losses.sparse_categorical_crossentropy(labels, outp))
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  return loss


@tf.function
def eval_step(inp, labels, model):
  outp = model(inp, training=False)
  pred = tf.cast(tf.argmax(outp, axis=1), tf.int32)
  n_correct = tf.reduce_sum(
    tf.cast(
      tf.equal(pred, labels),
      tf.float32
    )
  )
  acc = n_correct / tf.cast(tf.shape(outp)[0], tf.float32)
  return acc

Here we create an RNN called ```rnn_0``` using the RNN_model class from before.



In [ ]:
# Parameter save for convenience

inp = np.zeros([1, 21, 2], dtype=np.float32)
rnn_0 = RNN_model(32)
s = rnn_0(inp)

param_save = rnn_0.get_weights()

 #### QUESTION 2

Train the RNN for 10000 iterations; report training loss and validation acccuracy every 100 iterations.

In [ ]:
rnn_0.set_weights(param_save)

average_training_losses = []
training_losses = []
average_val_accs = []
val_accs = []

for id_iter, (x_train_in, y_train_in) in zip(range(10000), train_dataset.repeat()):
    # TODO: training

#### QUESTION 3

Run a prediction on a sample from the validation set, and preview the corresponding trajectory. Repeat as many times as you would like.

In [ ]:
# TODO

#### QUESTION 4

Modify BRNN_model to make it into a **bidirectional RNN**:
- one RNN processing the sequence in chronological order
- another processing it in reverse order
- concatenate RNN outputs

In [ ]:
class BRNN_model(tf.keras.Model):
  def __init__(self, memory_size):
    super().__init__()
    # TODO
    self._dense = tf.keras.layers.Dense(
      units=N_CLASSES,
      activation="softmax"
    )

  def call(self, x):
    res = x
    # TODO
    return res

Once the code for the bidirectional RNN class is ready, create the bidirectional RNN using the cell below:

In [ ]:
# Parameter save for convenience

inp = np.zeros([1, 21, 2], dtype=np.float32)
brnn_0 = BRNN_model(32)
s = brnn_0(inp)

param_save = brnn_0.get_weights()

#### QUESTION 5

Train the bidirectional RNN for 10000 iterations; report training loss and validation acccuracy every 100 iterations.

In [ ]:
# TODO

EXTRA QUESTION:

The following function takes an array of 2D coordinates over time and draws the corresponding trajectory in a grayscale (86, 86) image.

Convert the train and val dataset to trajectory images and try training a small CNN on the converted train set. How does the performance compare to the RNN?

In [ ]:
def to_img(positions):
    SZ = 86
    bbox = trajectory_bbox(positions)
    arr = np.zeros([SZ, SZ], dtype=np.float32)
    for p in positions:
        x = int((p[0] - bbox[0]) / (p[0] = bbox[1]) * SZ)
        y = int((p[1] - bbox[2]) / (p[1] = bbox[3]) * SZ)
        arr[y, x] = 1.0
    return arr